##### Copyright 2024 The AI Edge Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorFlow Lite Model Analyzer

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/lite/guide/model_analyzer"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/tensorflow/blob/master/tensorflow/lite/g3doc/guide/model_analyzer.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/tensorflow/blob/master/tensorflow/lite/g3doc/guide/model_analyzer.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/tensorflow/tensorflow/lite/g3doc/guide/model_analyzer.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

TensorFlow Lite Model Analyzer API helps you analyze models in TensorFlow Lite format by listing a model's structure.


## Model Analyzer API

The following API is available for the TensorFlow Lite Model Analyzer.

```
tf.lite.experimental.Analyzer.analyze(model_path=None,
                                      model_content=None,
                                      gpu_compatibility=False)
```

You can find the API details from https://www.tensorflow.org/api_docs/python/tf/lite/experimental/Analyzer or run `help(tf.lite.experimental.Analyzer.analyze)` from a Python terminal.


## Basic usage with simple Keras model

The following code shows basic usage of Model Analyzer. It shows contents of the converted Keras model in TFLite model content, formatted as a flatbuffer object.

In [1]:
import tensorflow as tf

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(128, 128)),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

fb_model = tf.lite.TFLiteConverter.from_keras_model(model).convert()

tf.lite.experimental.Analyzer.analyze(model_content=fb_model)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Saved artifact at '/tmp/tmpm4qo8ooh'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 128, 128), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  140338995377168: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140341634565904: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140338995379472: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140338995378320: TensorSpec(shape=(), dtype=tf.resource, name=None)
=== TFLite ModelAnalyzer ===

Your TFLite model has '1' subgraph(s). In the subgraph description below,
T# represents the Tensor numbers. For example, in Subgraph#0, the RESHAPE op takes
tensor #0 and tensor #3 as input and produces tensor #4 as output.

Subgraph#0 main(T#0) -> [T#6]
  Op#0 RESHAPE(T#0, T#3[-1, 16384]) -> [T#4]
  Op#1 FULLY_CONNECTED(T#4, T#2, T#-1) -> [T#5]
  Op#2 FULLY_CONNECTED(T#5, T#1, T#-1) -> [T#6]

Tensors of 

## Basic usage with MobileNetV3Large Keras model

This API works with large models such as MobileNetV3Large. Since the output is large, you might want to browse it with your favorite text editor.

In [2]:
model = tf.keras.applications.MobileNetV3Large()
fb_model = tf.lite.TFLiteConverter.from_keras_model(model).convert()

tf.lite.experimental.Analyzer.analyze(model_content=fb_model)

/usr/local/lib/python3.12/dist-packages/keras/src/applications/mobilenet_v3.py:517: UserWarning: `input_shape` is undefined or non-square, or `rows` is not 224. Weights for input shape (224, 224) will be loaded as the default.
  return MobileNetV3(


22661472/22661472 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
Saved artifact at '/tmp/tmp0ihhdgns'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, None, None, 3), dtype=tf.float32, name='keras_tensor_5')
Output Type:
  TensorSpec(shape=(None, 1000), dtype=tf.float32, name=None)
Captures:
  140338992732624: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140338992732432: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140338992734160: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140338992734544: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140338992734352: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140338992733584: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140338992735504: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140338992735312: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140338992732816: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140338992736080: TensorSpe

## Check GPU delegate compatibility

The ModelAnalyzer API provides a way to check the [GPU delegate](https://www.tensorflow.org/lite/performance/gpu) compatibility of the given model by providing `gpu_compatibility=True` option.


### Case 1: When model is incompatibile

The following code shows a way to use `gpu_compatibility=True` option for simple tf.function which uses `tf.slice` with a 2D tensor and `tf.cosh` which are not compatible with GPU delegate.

You will see `GPU COMPATIBILITY WARNING` per every node which has compatibility issue(s).

In [3]:
import tensorflow as tf

@tf.function(input_signature=[
    tf.TensorSpec(shape=[4, 4], dtype=tf.float32)
])
def func(x):
  return tf.cosh(x) + tf.slice(x, [1, 1], [1, 1])

converter = tf.lite.TFLiteConverter.from_concrete_functions(
    [func.get_concrete_function()], func)
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,
    tf.lite.OpsSet.SELECT_TF_OPS,
]
fb_model = converter.convert()

tf.lite.experimental.Analyzer.analyze(model_content=fb_model, gpu_compatibility=True)

=== TFLite ModelAnalyzer ===

Your TFLite model has '1' subgraph(s). In the subgraph description below,
T# represents the Tensor numbers. For example, in Subgraph#0, the FlexCosh op takes
tensor #0 as input and produces tensor #2 as output.

Subgraph#0 main(T#0) -> [T#4]
  Op#0 FlexCosh(T#0) -> [T#2]
  Op#1 SLICE(T#0, T#1[1, 1], T#1[1, 1]) -> [T#3]
  Op#2 ADD(T#2, T#3) -> [T#4]

Tensors of Subgraph#0
  T#0(x) shape:[4, 4], type:FLOAT32
  T#1(arith.constant) shape:[2], type:INT32 RO 8 bytes, buffer: 2, data:[1, 1]
  T#2(Cosh) shape:[4, 4], type:FLOAT32
  T#3(Slice) shape:[1, 1], type:FLOAT32
  T#4(Identity) shape:[4, 4], type:FLOAT32


Your model looks compatible with GPU delegate on TFLite runtime version 2.19.0.
This does not guarantee that your model will work well with GPU delegate because there could still be runtime incompatibililties.
---------------------------------------------------------------
              Model size:       1140 bytes
    Non-data buffer size:       1012 byt

### Case 2: When model is compatibile

In this example, the given model is compatbile with GPU delegate.

**Note:** Even though the tool doesn't find any compatibility issue, it doesn't guarantee that your model works well with GPU delegate on every device. There could be some runtime incompatibililty happen such as missing `CL_DEVICE_IMAGE_SUPPORT` feature by target OpenGL backend.


In [4]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(128, 128)),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

fb_model = tf.lite.TFLiteConverter.from_keras_model(model).convert()

tf.lite.experimental.Analyzer.analyze(model_content=fb_model, gpu_compatibility=True)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Saved artifact at '/tmp/tmpmm5_mtl_'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 128, 128), dtype=tf.float32, name='keras_tensor_215')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  140336850921296: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140336850922064: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140336850923024: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140336850922256: TensorSpec(shape=(), dtype=tf.resource, name=None)
=== TFLite ModelAnalyzer ===

Your TFLite model has '1' subgraph(s). In the subgraph description below,
T# represents the Tensor numbers. For example, in Subgraph#0, the RESHAPE op takes
tensor #0 and tensor #3 as input and produces tensor #4 as output.

Subgraph#0 main(T#0) -> [T#6]
  Op#0 RESHAPE(T#0, T#3[-1, 16384]) -> [T#4]
  Op#1 FULLY_CONNECTED(T#4, T#2, T#-1) -> [T#5]
  Op#2 FULLY_CONNECTED(T#5, T#1, T#-1) -> [T#6]

Tensors